<a href="https://colab.research.google.com/github/savindumahasen/Skin-Disease-Classification-Project/blob/main/Skin_disease_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## install the necessary libraries
!pip install numpy -q
!pip install pandas -q
!pip install matplotlib -q
!pip install tensorflow -q
!pip install opendatasets -q

In [2]:
## Import the necessary libraries
import opendatasets as od
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [3]:
## download the dataset
od.download("https://www.kaggle.com/datasets/pritpal2873/multiple-skin-disease-detection-and-classification")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: savinduruhunuhewa
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/pritpal2873/multiple-skin-disease-detection-and-classification


100%|██████████| 785M/785M [00:06<00:00, 119MB/s]


In [4]:
## Define the batch size and image size
BATCH_SIZE=32
IMAGE_SIZE=(128,128)

In [5]:
import os
import random
import shutil
from math import floor
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Define paths
dataset_dir = '/content/multiple-skin-disease-detection-and-classification/Skin Cancer Dataset'
output_dir = '/content/multiple-skin-disease-detection-and-classification/working'

# Allowed image extensions
IMG_EXTENSIONS = ('.png', '.jpg', '.jpeg', '.bmp')

def split_dataset(dataset_dir, output_dir, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    train_dir = os.path.join(output_dir, 'Train')
    val_dir = os.path.join(output_dir, 'Validation')
    test_dir = os.path.join(output_dir, 'Test')

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    for class_name in os.listdir(dataset_dir):
        class_dir = os.path.join(dataset_dir, class_name)
        if not os.path.isdir(class_dir):
            continue

        image_files = [
            f for f in os.listdir(class_dir)
            if f.lower().endswith(IMG_EXTENSIONS) and os.path.isfile(os.path.join(class_dir, f))
        ]
        random.shuffle(image_files)

        total_images = len(image_files)
        train_count = floor(total_images * train_ratio)
        val_count = floor(total_images * val_ratio)
        test_count = total_images - train_count - val_count

        train_files = image_files[:train_count]
        val_files = image_files[train_count:train_count + val_count]
        test_files = image_files[train_count + val_count:]

        for file in train_files:
            src = os.path.join(class_dir, file)
            dest = os.path.join(train_dir, class_name)
            os.makedirs(dest, exist_ok=True)
            shutil.copy(src, dest)

        for file in val_files:
            src = os.path.join(class_dir, file)
            dest = os.path.join(val_dir, class_name)
            os.makedirs(dest, exist_ok=True)
            shutil.copy(src, dest)

        for file in test_files:
            src = os.path.join(class_dir, file)
            dest = os.path.join(test_dir, class_name)
            os.makedirs(dest, exist_ok=True)
            shutil.copy(src, dest)

    print("✅ Dataset successfully split into Train, Validation, and Test sets.\n")


# ✅ Step 1: Split dataset
split_dataset(dataset_dir, output_dir)

# ✅ Step 2: Display images from Validation set using matplotlib (Colab compatible)
val_dir = os.path.join(output_dir, 'Validation')
print("📂 Reading images from Validation set...\n")

for class_name in os.listdir(val_dir):
    class_path = os.path.join(val_dir, class_name)
    if not os.path.isdir(class_path):
        continue

    print(f"🧪 Class: {class_name}")
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)

        # Filter only image files
        #if os.path.isfile(img_path) and img_name.lower().endswith(IMG_EXTENSIONS):
        #    try:
        #        img = mpimg.imread(img_path)
        #        plt.imshow(img)
        #        plt.title(f"Class: {class_name}")
        #        plt.axis('off')
        #        plt.show()
        #    except Exception as e:
        #        print(f"⚠️ Failed to load image {img_path}: {e}")


✅ Dataset successfully split into Train, Validation, and Test sets.

📂 Reading images from Validation set...

🧪 Class: Melanoma
🧪 Class: Squamous Cell Carcinoma
🧪 Class: Dermatofibroma
🧪 Class: Nevus
🧪 Class: Acitinic Keratosis
🧪 Class: Basal Cell Carcinoma
🧪 Class: Pigmented Benign Keratosis
🧪 Class: Vascular Lesion
🧪 Class: Seborrheic Keratosis


In [10]:
## Load the train set, test set, and validation set
train_dir = "/content/multiple-skin-disease-detection-and-classification/working/Train"
test_dir="/content/multiple-skin-disease-detection-and-classification/working/Test"
validation_dir="/content/multiple-skin-disease-detection-and-classification/working/Validation"

train_set=tf.keras.utils.image_dataset_from_directory(train_dir,
                                                      batch_size=BATCH_SIZE,
                                                      image_size=IMAGE_SIZE)
test_dir=tf.keras.utils.image_dataset_from_directory(test_dir,
                                                     batch_size=BATCH_SIZE,
                                                     image_size=IMAGE_SIZE)
validation_dir=tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                          batch_size=BATCH_SIZE,
                                                          image_size=IMAGE_SIZE)

Found 3285 files belonging to 9 classes.
Found 414 files belonging to 9 classes.
Found 408 files belonging to 9 classes.


In [ ]:
## Look at the one batch
for image_size, label_batch in train_set.take(1):
  print(image_size.shape)
  print(label_batch.shape)

In [ ]:
## Plot the training set sample

plt.figure(figsize=(10,4))
for image,label in train_set.take(1):
  for i in range(10):
    ax=plt.subplot(2,5,i+1)
    plt.imshow(image[i].numpy().astype("uint8"))
    plt.title(class_names[label[i]], fontsize=6)
    plt.axis('off')

In [ ]:
## Look at the images pixal values by taking one batch
for image, label in train_set.take(1):
  print(image)

In [ ]:
## Normalize the  train and test dataset
train_set_new=train_set.map(lambda x,y:(x/255,y))
#test_set_new=test_set.map(lambda x,y:(x/255,y))

In [ ]:
## Print the normalize data of one batch of train set
#for image, label in train_set_new.take(1):
#    print(image)

In [ ]:
## use the pre-train  Xception model
#pretrained_model=tf.keras.applications.Xception(include_top=False,
#                               input_shape=(128,128,3),
#                               weights='imagenet',
#                               pooling='max',
#                                               )
#pretrained_model

#for layer in pretrained_model.layers:
#  layer.trainable=False


#pretrained_model=tf.keras.applications.ResNet50(include_top=False,
#                                                weights='imagenet',
#                                                input_shape=(128,128,3),
#                                                pooling='max',

#)

In [ ]:
## Build the model

model=tf.keras.models.Sequential()
#model.add(pretrained_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(32,activation='relu'))
model.add(tf.keras.layers.Dense(64,activation='relu'))
model.add(tf.keras.layers.Dense(128,activation='relu'))

model.add(tf.keras.layers.Dense(9,activation='softmax'))


In [ ]:
## get the summary of model

model.summary()

In [ ]:
## compile the model


model.compile(optimizer=tf.keras.optimizers.SGD(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['accuracy'])

In [ ]:
## train the model
import time
start_time=time.time()
history=model.fit(train_set_new,validation_data=validation_set, epochs=10, batch_size=32)
end_time=time.time()


In [ ]:

## plot the  accuracy  and loss
fig=plt.figure()
plt.plot(history.history['accuracy'], color='teal',label='accuracy')
plt.plot(history.history['loss'], color='orange', label='loss')
plt.suptitle('Accuracy vs Loss', fontsize=20)
plt.legend()
plt.show()

In [ ]:
## Test set

In [ ]:
## Plot the testing set sample

plt.figure(figsize=(10,4))
for image,label in test_set.take(1):
  for i in range(10):
    ax=plt.subplot(2,5,i+1)
    plt.imshow(image[i].numpy().astype("uint8"))
    plt.title(class_names[label[i]], fontsize=6)
    plt.axis('off')

In [ ]:
## Get the test data image and its label shape
def test_image():
  for image,label in test_set:
    return image
test_image=test_image()
print(test_image.shape)


def test_image_label():
  for image,label in test_set:
    return label
test_label=test_image_label()
print(test_label.shape)

In [ ]:
## import the model peformance metrices

precision=tf.keras.metrics.Precision()
recall=tf.keras.metrics.Recall()
accuracy=tf.keras.metrics.Accuracy()

In [ ]:
## make the predictions using  one test data batch
model_predictions_new=model.predict(test_image)
model_predictions_new.shape

In [ ]:

## evaluvate the model performance using performance metrices

for batch in test_set.as_numpy_iterator():
    X, y = batch

    yhat = model.predict(X)  # (32, 9)
    yhat_classes = np.argmax(yhat, axis=1)  # (32,)

    precision.update_state(y, yhat_classes)
    recall.update_state(y,yhat_classes)
    accuracy.update_state(y,yhat_classes)

In [ ]:
precision.result(), recall.result(), accuracy.result()

In [ ]:
## Make the predictions based on the test data
predictions = model.predict(X)
predictions[1]

In [ ]:
class_names

In [ ]:

## get the predicted class  index
import numpy  as np
predicted_index=np.argmax(predictions[1])
predicted_index

In [ ]:

class_name=['Actinic Keratosis','Basal Cell Carcinoma','Dermatofibroma', 'Melanoma','Nevus','Pigmented Benign Keratosis','Seborrheic Keratosis','Squamous Cell Carcinoma','Vascular Lesion']
predicted_class=class_name[predicted_index]
print("Brain Tumor : ",predicted_class)

In [ ]:
## use the model based on the user data

In [ ]:
!pip install opencv-python

In [ ]:
import cv2

In [ ]:

image=cv2.imread("/content/multiple-skin-disease-detection-and-classification/Skin Cancer Dataset/Acitinic Keratosis/ISIC_7199177.jpg")
plt.imshow(image)
plt.show()

In [ ]:
## scaled the data

resized_image=tf.image.resize(image,IMAGE_SIZE)
scaled_image=resized_image/255

In [ ]:
scaled_image